In [ ]:

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from skimage.io import imread, imshow


from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV


from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from keras.preprocessing.image import ImageDataGenerator

from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

import tensorflow as tf
# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
                  
from random import shuffle  
from zipfile import ZipFile
from PIL import Image
from skimage import filters, feature
from skimage.filters import threshold_local



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
#to get the list of all files and directories in the specified directory
print(os.listdir('../input/flowers-recognition/flowers'))

In [ ]:
FLOWER_DAISY_DIR='../input/flowers-recognition/flowers/daisy'
FLOWER_SUNFLOWER_DIR='../input/flowers-recognition/flowers/sunflower'
FLOWER_TULIP_DIR='../input/flowers-recognition/flowers/tulip'
FLOWER_DANDI_DIR='../input/flowers-recognition/flowers/dandelion'
FLOWER_ROSE_DIR='../input/flowers-recognition/flowers/rose'

In [ ]:
def assign_label(img,flower_type):
    return flower_type

In [ ]:
import cv2                  
import numpy as np  

In [ ]:
X=[]
Z=[] # 5 Classes
IMG_SIZE=150

def make_train_data(flower_type,DIR):
    for img in os.listdir(DIR):
        label=assign_label(img,flower_type)
        path = os.path.join(DIR,img)
        
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        X.append(np.array(img))  # Create Numpy array of images
        Z.append(str(label))

In [ ]:
make_train_data('Daisy',FLOWER_DAISY_DIR)
print(len(X))

In [ ]:
make_train_data('Sunflower',FLOWER_SUNFLOWER_DIR)
print(len(X))

In [ ]:
make_train_data('Tulip',FLOWER_TULIP_DIR)
print(len(X))

In [ ]:
make_train_data('Dandelion',FLOWER_DANDI_DIR)
print(len(X))

In [ ]:
make_train_data('Rose',FLOWER_ROSE_DIR)
print(len(X))

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [ ]:
# convert categorical to numerical values.
label=LabelEncoder()
Y=label.fit_transform(Z)

# Converts a class vector (numerical) to binary class matrix.
Y=to_categorical(Y,5)
X=np.array(X)
X=X/255


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=42)

In [ ]:
model = Sequential()
# convolution kernel
# 'zero-padded' The output volume size matches the input volume size
# input_shape which means the images the images are 150 x 150 pixels, with 3 channels
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (150,150,3)))
# Max Pooling to reduce the spatial dimensions of the output volume
# (2, 2) will halve the image in each dimension
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
 

model.add(Conv2D(filters =96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

'''
Note that the number of filters grows as we climb up the CNN towards the out‐
put layer (it is initially 64, then 128, then 256): it makes sense for it to grow, since
the number of low level features is often fairly low (e.g., small circles, horizontal
lines, etc.), but there are many different ways to combine them into higher level
features. It is a common practice to double the number of filters after each pool‐
ing layer: since a pooling layer divides each spatial dimension by a factor of 2, we
can afford doubling the number of feature maps in the next layer, without fear of
exploding the number of parameters, memory usage, or computational load.

'''
model.add(Flatten())
model.add(Dense(512), activation='relu')
model.add(Dense(5, activation = "softmax"))

In [ ]:
batch_size=128
epochs=50

from keras.callbacks import ReduceLROnPlateau
red_lr= ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.1)

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

In [ ]:
model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
History = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_test,y_test),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size)

In [ ]:
plt.plot(History.history['accuracy'])
plt.plot(History.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
# getting predictions on val set.
results = model.evaluate(x_test, y_test, batch_size=128)

#print("test acc:", int(results[1]*100),"%")

pred=model.predict(x_test)
pred_digits=np.argmax(pred,axis=1)


In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print(classification_report(y_test, pred))
print(confusion_matrix(y_test, pred))